# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [1]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

Setting path to train and test data.

In [2]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32

Loading train data.

In [3]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [4]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [5]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [6]:
labels, images = load_train_data(TRAIN_DATA_DIR)
test_labels, test_images = load_test_data(TEST_DATA_DIR)

In [7]:
print("Total training images {0}".format(len(images)))
print("Total testing images {0}".format(len(test_images)))

Total training images 42890
Total testing images 12630


Creating bar chart for dataset Visualization.

In [ ]:
data = {x:labels.count(x) for x in labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel("Number of examples")
plt.title("Training dataset")
plt.show()

Resize and normalize images.

In [ ]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

Difference between normal image and a normalized one.

In [ ]:
plt.figure(figsize = (10, 10))
plt.subplot(1, 2, 1)
plt.axis('off')
plt.title("Normal image")  
plt.imshow(images[1800])
plt.subplot(1, 2, 2)
plt.axis('off')
plt.title("Normalized image")
plt.imshow(normalize(images[1800]))
plt.show()

In [8]:
with tf.device('/device:GPU:0'):
    images_ph = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3], name = "images_ph")
    labels_ph = tf.placeholder(tf.int64, [None])

    images_flat = tf.contrib.layers.flatten(images_ph)

    logits = tf.contrib.layers.fully_connected(images_flat, len(set(labels)), tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1, name="prediction")
   
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels_ph)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels_ph)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()

In [9]:
saver = tf.train.Saver()
session = tf.Session()
session.run(init)

In [10]:
labels_arr = np.array(labels)
images_arr = np.array(images)

In [11]:
feed_dict_train = {images_ph: images_arr,
             labels_ph: labels_arr}
feed_dict_test = {images_ph: test_images,
             labels_ph: test_labels}

In [12]:
for i in range(10001):
    loss_value = session.run([optimizer, loss], feed_dict_train)
    if i % 10 == 0:
        print("Loss: ", loss_value)
    if i % 500 == 0:
        saver.save(session, './model/model', global_step = i)


score = session.run(accuracy, feed_dict_test)
print("Accuracy: {:.3f}".format(score))

('Loss: ', [None, 3.9706779])
('Loss: ', [None, 3.1857681])
('Loss: ', [None, 2.8908522])
('Loss: ', [None, 2.6617482])
('Loss: ', [None, 2.4879014])
('Loss: ', [None, 2.351897])
('Loss: ', [None, 2.2436535])
('Loss: ', [None, 2.1538067])
('Loss: ', [None, 2.0772076])
('Loss: ', [None, 2.0103598])
('Loss: ', [None, 1.9508309])
('Loss: ', [None, 1.8967865])
('Loss: ', [None, 1.8464411])
('Loss: ', [None, 1.7921059])
('Loss: ', [None, 1.745193])
('Loss: ', [None, 1.704031])
('Loss: ', [None, 1.6666102])
('Loss: ', [None, 1.6326324])
('Loss: ', [None, 1.6014235])
('Loss: ', [None, 1.5724291])
('Loss: ', [None, 1.5453323])
('Loss: ', [None, 1.51989])
('Loss: ', [None, 1.4959053])
('Loss: ', [None, 1.4732188])
('Loss: ', [None, 1.4516982])
('Loss: ', [None, 1.4312214])
('Loss: ', [None, 1.4116684])
('Loss: ', [None, 1.3929033])
('Loss: ', [None, 1.3746281])
('Loss: ', [None, 1.3567135])
('Loss: ', [None, 1.3394922])
('Loss: ', [None, 1.3232489])
('Loss: ', [None, 1.3075868])
('Loss: ', [Non

('Loss: ', [None, 0.49593282])
('Loss: ', [None, 0.49507412])
('Loss: ', [None, 0.49421981])
('Loss: ', [None, 0.49336994])
('Loss: ', [None, 0.49252439])
('Loss: ', [None, 0.49168313])
('Loss: ', [None, 0.49084622])
('Loss: ', [None, 0.49001348])
('Loss: ', [None, 0.48918492])
('Loss: ', [None, 0.48836058])
('Loss: ', [None, 0.48754022])
('Loss: ', [None, 0.48672399])
('Loss: ', [None, 0.48591176])
('Loss: ', [None, 0.48510352])
('Loss: ', [None, 0.48429921])
('Loss: ', [None, 0.4834989])
('Loss: ', [None, 0.48270243])
('Loss: ', [None, 0.48191011])
('Loss: ', [None, 0.48113716])
('Loss: ', [None, 0.48034379])
('Loss: ', [None, 0.47956187])
('Loss: ', [None, 0.4787865])
('Loss: ', [None, 0.47800952])
('Loss: ', [None, 0.47724178])
('Loss: ', [None, 0.47647676])
('Loss: ', [None, 0.47571549])
('Loss: ', [None, 0.47495797])
('Loss: ', [None, 0.47420391])
('Loss: ', [None, 0.47345334])
('Loss: ', [None, 0.4727062])
('Loss: ', [None, 0.47196248])
('Loss: ', [None, 0.47122213])
('Loss: ', 

('Loss: ', [None, 0.35725769])
('Loss: ', [None, 0.35699326])
('Loss: ', [None, 0.35659027])
('Loss: ', [None, 0.35629687])
('Loss: ', [None, 0.35597071])
('Loss: ', [None, 0.3556636])
('Loss: ', [None, 0.35535678])
('Loss: ', [None, 0.35505146])
('Loss: ', [None, 0.35474735])
('Loss: ', [None, 0.35444397])
('Loss: ', [None, 0.35414132])
('Loss: ', [None, 0.3538394])
('Loss: ', [None, 0.35353819])
('Loss: ', [None, 0.35323772])
('Loss: ', [None, 0.35293797])
('Loss: ', [None, 0.35263887])
('Loss: ', [None, 0.35234046])
('Loss: ', [None, 0.35204273])
('Loss: ', [None, 0.35174567])
('Loss: ', [None, 0.35144925])
('Loss: ', [None, 0.35115337])
('Loss: ', [None, 0.35085818])
('Loss: ', [None, 0.35056353])
('Loss: ', [None, 0.35026953])
('Loss: ', [None, 0.34997618])
('Loss: ', [None, 0.34968391])
('Loss: ', [None, 0.34942126])
('Loss: ', [None, 0.34930196])
('Loss: ', [None, 0.34882203])
('Loss: ', [None, 0.34856001])
('Loss: ', [None, 0.34825194])
('Loss: ', [None, 0.34797004])
('Loss: ',

('Loss: ', [None, 0.26948208])
('Loss: ', [None, 0.26933157])
('Loss: ', [None, 0.26918194])
('Loss: ', [None, 0.2690331])
('Loss: ', [None, 0.26888511])
('Loss: ', [None, 0.26873782])
('Loss: ', [None, 0.26859123])
('Loss: ', [None, 0.26844531])
('Loss: ', [None, 0.26830003])
('Loss: ', [None, 0.2681554])
('Loss: ', [None, 0.26801136])
('Loss: ', [None, 0.2678737])
('Loss: ', [None, 0.26782739])
('Loss: ', [None, 0.26759109])
('Loss: ', [None, 0.26744714])
('Loss: ', [None, 0.26730877])
('Loss: ', [None, 0.26716757])
('Loss: ', [None, 0.26702845])
('Loss: ', [None, 0.26689076])
('Loss: ', [None, 0.2667532])
('Loss: ', [None, 0.26661614])
('Loss: ', [None, 0.26647943])
('Loss: ', [None, 0.26634309])
('Loss: ', [None, 0.26620707])
('Loss: ', [None, 0.26607138])
('Loss: ', [None, 0.26593593])
('Loss: ', [None, 0.26580068])
('Loss: ', [None, 0.26566556])
('Loss: ', [None, 0.26553038])
('Loss: ', [None, 0.26539457])
('Loss: ', [None, 0.26525638])
('Loss: ', [None, 0.26510203])
('Loss: ', [

In [13]:
saver.save(session, './model/model')

'./model/model'

In [14]:
session.close()